In [2]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from stop_words import get_stop_words
from nltk.stem.snowball import SnowballStemmer
from gensim import corpora, models
import gensim

In [3]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/s2191989/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
import findspark
findspark.init()

In [6]:
import pyspark

In [7]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

In [8]:
sc = SparkContext(appName="SDDM")

In [9]:
sc = SparkContext.getOrCreate()

In [10]:
sc.master

'spark://fs.das3:7077'

In [11]:
from pyspark.sql import SparkSession

In [12]:
ss = SparkSession.builder.getOrCreate()

In [18]:
import pandas as pd

In [62]:
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield (eval(l))
        
def GetDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient = "index")

df = GetDF("reviews_Sports_and_Outdoors_5.json.gz")

df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AIXZKN4ACSKI,1881509818,David Briner,"[0, 0]",This came in on time and I am veru happy with ...,5.0,Woks very good,1390694400,"01 26, 2014"
1,A1L5P841VIO02V,1881509818,Jason A. Kramer,"[1, 1]",I had a factory Glock tool that I was using fo...,5.0,Works as well as the factory tool,1328140800,"02 2, 2012"
2,AB2W04NI4OEAD,1881509818,J. Fernald,"[2, 2]",If you don't have a 3/32 punch or would like t...,4.0,"It's a punch, that's all.",1330387200,"02 28, 2012"
3,A148SVSWKTJKU6,1881509818,"Jusitn A. Watts ""Maverick9614""","[0, 0]",This works no better than any 3/32 punch you w...,4.0,It's a punch with a Glock logo.,1328400000,"02 5, 2012"
4,AAAWJ6LW9WMOO,1881509818,Material Man,"[0, 0]",I purchased this thinking maybe I need a speci...,4.0,"Ok,tool does what a regular punch does.",1366675200,"04 23, 2013"


In [65]:
df = df[['reviewText', 'summary']]

In [67]:
sp_df = SQLContext(sc).createDataFrame(df)

In [72]:
sp_df = sp_df.na.drop()

In [73]:
sp_df.show(5)

+--------------------+--------------------+
|          reviewText|             summary|
+--------------------+--------------------+
|This came in on t...|      Woks very good|
|I had a factory G...|Works as well as ...|
|If you don't have...|It's a punch, tha...|
|This works no bet...|It's a punch with...|
|I purchased this ...|Ok,tool does what...|
+--------------------+--------------------+
only showing top 5 rows



In [74]:
sp_df.select('reviewText').first()

Row(reviewText='This came in on time and I am veru happy with it, I haved used it already and it makes taking out the pins in my glock 32 very easy')

In [82]:
rdd = sp_df.select('reviewText').rdd

In [76]:
tokenizer = RegexpTokenizer(r'\w+')
nltk_stpwd = stopwords.words("english")
stop_words_stpwd = get_stop_words('en')
merged_stopwords = list(set(nltk_stpwd + stop_words_stpwd))
sb_stemmer = SnowballStemmer('english')

In [77]:
def dropStopWords(s):
    tokens = tokenizer.tokenize(s[0])
    stopped_tokens = [token for token in tokens if token not in merged_stopwords]
    stemmed_tokens = [sb_stemmer.stem(token) for token in stopped_tokens]
    
    # add tokens to list
    return(stemmed_tokens)

In [84]:
texts_rdd = rdd.flatMap(dropStopWords)